# Generosity and Social Support in the pandemic. 
In this Data visualization I want to look at how the pandemic has affected people perciotion of Generosity and Social Support in the past year. I am basing myself on the assumption that 2020 and 2021 are considered "pandemic years" and all previous years are not considered pandemic years.


Hypothoisis: Before I start the visualization. I have some general assumptions of what might play out. 

For generosity I believe there will be an over all decline in generosity over all but I beleive the wealtheir countries will have a steeper decline than poor countries. I also believe less happy contries will see less of a change in generosity in comparison to a downward slope for happier countries.

For social support, my assumption is that it will have grown for both happy and less happy countries. But I assume wealthier countries will see a dip in soical support when comparing to poorer countries. 

In [ ]:
#import all libraries
!pip install plotly &> /dev/null
import plotly
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express  as px
import plotly.graph_objs as go
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot


In [ ]:
#bring in data
report21 = pd.read_csv('../input/world-happiness-report-2021/world-happiness-report-2021.csv')
report = pd.read_csv('../input/world-happiness-report-2021/world-happiness-report.csv')
df_report = pd.DataFrame(report)
df_report21 = pd.DataFrame(report21)

In [ ]:
#modifying column names of 2021 to fit the other report
df_report21 = df_report21.rename(columns={'Country name': 'Country name',
                                         'Ladder score': 'Life Ladder',
                                         'Logged GDP per capita':'Log GDP per capita' ,
                                         'Healthy life expectancy':'Healthy life expectancy at birth'})
plotly.__version__

First Let's start by understanding each column:

 0.   Country name.                      
 1.   Year: year in which country data was taken                            
 2.   Life Ladder: I could not find the direct definition for this one but it seems to align with the  “Cantril Ladder” which is:  The main life evaluation question asked in the poll is: “Please imagine a ladder, with steps numbered from 0 at the bottom to 10 at the top. The top of the ladder represents the best possible life for you and the bottom of the ladder represents the worst possible life for you. On which step of the ladder would you say you personally feel you stand at this time?”
 3.   Log GDP per capita: The GDP-per-capita time series from 2019 to 2020 using country specific forecasts of real GDP growth in 2020.
 4.   Social support: based on yes(1) or no(0) to the answer: "If you were in trouble do you think your family or friends would help?"                    
 5.   Healthy life expectancy at birth. 
 6.   Freedom to make life choices: based on yes(1) or no(0) to the answer:"Are you satisfied or dissatisfied with the freedom to choose what you do with your life?"
 7.   Generosity: Residual of regressing the national average of GWP responses to: "Have you donated money to a charity in the past month?" on a GDP per capita.                      
 8.   Perceptions of corruption: based on yes(1) or no(0) to the answer:"Is corruption widespread within the government or not?" and "Is corruption widespread within the businesses or not?" 
 9.   Positive affect: is defined as the average of previous-day affect measures for happiness, laughter, and enjoyment for certain waves in the GWP, sometimes this question is not asked. The general form for the question is: "Did you experience the following feelings during a lot of yesterday?"            
 10.  Negative affect: Same thing as above but instead for: worry, sadness and anger.

 Source: https://worldhappiness.report/ed/2020/social-environments-for-world-happiness/

 https://ourworldindata.org/happiness-and-life-satisfaction
# Shaping Tables

In [ ]:
#Ading all rows with 2020 to a new column
pandemic_filter = df_report['year']== 2020
df_pandemic = df_report[pandemic_filter]
df_pandemic = pd.concat([df_pandemic, df_report21], ignore_index=True)
df_report = df_report[~pandemic_filter]

#Adding 2021 to the years
df_pandemic['year']=df_pandemic['year'].replace(np.nan, 2021)

#Putting rows in order
df_pandemic = df_pandemic.sort_values(by = ['Country name', 'year'])
df_pandemic = df_pandemic.reset_index()
df_pandemic = df_pandemic.drop(columns=['index'])

#Add  Regional indicator to all rows
for index, row in df_pandemic.iterrows():
  if df_pandemic.iloc[index]['year'] == 2021 and df_pandemic.iloc[index]['Country name'] == df_pandemic.iloc[index-1]['Country name']:
    df_pandemic.loc[index-1, 'Regional indicator'] = df_pandemic.iloc[index]['Regional indicator']

df_pandemic = df_pandemic.drop(columns=['Standard error of ladder score',
                              'upperwhisker',
                              'lowerwhisker',
                              'Ladder score in Dystopia',
                              'Explained by: Log GDP per capita',
                              'Explained by: Social support',
                              'Explained by: Freedom to make life choices', 
                              'Explained by: Generosity',
                              'Explained by: Perceptions of corruption',
                              'Explained by: Healthy life expectancy',
                              'Dystopia + residual'])

In [ ]:
#Combing both table for situations wherer visualizing all the data is needed
df_all_reports = pd.concat([df_pandemic, df_report])
#Putting rows in order
df_all_reports = df_all_reports.sort_values(by = ['Country name', 'year'])
df_all_reports = df_all_reports.reset_index()
df_all_reports = df_all_reports.drop(columns=['index'])

As you can see below there is missing data for the columns we will be looking at. Let display what Generoisity is missing on a map.

In [ ]:
df_all_reports.describe()

# Visulization Of Generosity
The first thing that is very noticable is "generosity" looks like it will have decreased over all. lets see if that is indeed the case.

In [ ]:
scl = [[0.0, '#782102'],[0.2, '#ff5517'],[0.4, '#ffddd1'], [0.6, '#00e64d'],[0.8, '#009933'],[1.0, '#003311']] # greens

In [ ]:
years = df_all_reports['year'].unique()
country_name = df_all_reports.loc[:, 'Country name'].unique()
generosity_number = df_all_reports.loc[:, 'Generosity'].values
data_slider = []
data_one_year =[]
year1 = list(range(2005,2022))
for year in year1:
    df_gen_dic = df_all_reports[(df_all_reports['year']== year )]

    for col in df_all_reports.columns:  # I transform the columns into string type so I can:
        df_gen_dic[col] = df_gen_dic[col].astype(str)

    df_gen_dic['text'] = ''
    #Dictionary with the data for the current year
    data_one_year = dict(
                        type='choropleth',
                        locations = df_gen_dic['Country name'],
                        z=df_gen_dic['Generosity'].astype(float),
                        locationmode='country names',
                        colorscale = scl,
                        name='Generosity',)

    data_slider.append(data_one_year) # I add the dictionary to the list of dictionaries for the slider

In [ ]:
steps = []

for i in range(len(data_slider)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                label='Year {}'.format(i + 2005)) # label to be displayed for each step (year)
    step['args'][1][i] = True
    steps.append(step)

#Sliders' object from the 'steps' 
sliders = [dict(active=0, pad={"t": 1}, steps=steps)]  

In [ ]:
# I set up the layout (including slider option)

layout = dict(geo=dict(scope='world',
                        showocean=True,oceancolor ='#8c8c8c',
                        showcoastlines=True, coastlinecolor="#404040",
                        showland=True, landcolor="#bfbfbf",
                        showlakes=True, lakecolor="#cccccc",
                        projection={'type': 'natural earth'}, ),
              sliders=sliders)

fig = dict(data=data_slider, layout=layout,  ) 
plotly.offline.iplot(fig)
# The code for the map is based off: https://amaral.northwestern.edu/blog/step-step-how-plot-map-slider-represent-time-evolu

As you can see there is missing data in each year.
The Map above shows the generosity of countries over time. Overall there seems to be a decrease in the most recent years on the perception of generosity in most countries.

In [ ]:
df_above_mean= df_all_reports[ df_all_reports['Log GDP per capita'] >= 9.373]
df_below_mean= df_all_reports[ df_all_reports['Log GDP per capita'] < 9.373]

In [ ]:
figbox= go.Figure()
figbox.add_trace(go.Box(x= df_all_reports['year'], 
                    y = df_above_mean['Generosity'],
                    name='Above Mean',
                   marker_color='#248f24'))

figbox.add_trace(go.Box(x= df_all_reports['year'], 
                    y = df_below_mean['Generosity'],
                    name='Below Mean',
                    marker_color='#ff5517'))

figbox.update_xaxes(title_text = "Year",)
figbox.update_yaxes(showgrid=True, 
                        gridwidth=1, 
                        gridcolor='#f2f2f2', 
                        title_text = "Generosity",)


figbox.update_layout(
    title = 'Generosity of wealtheir and less wealthy countries.', boxmode='group',
    yaxis=dict(showgrid=True,zeroline=True, ),
    margin=dict(l=40, r=30, b=80, t=100,),
    plot_bgcolor='#e6e6e6',)

figbox.show()

The box plot above shows the generosity based on LOG GDP. Placing half the countries above the mean LOG GDP (9.373) and half below. My hypothesis was wrong in the assumption that generosity will decrease for all the wealthy or poor countries at around the same amount. 
 
Since 2010 the countries below the mean have been around zero, mainly slightly under it, with the lowest before the pandemic years being  -0.019 but during 2020 the median dipped much lower, to -0.032. Just the previous year in 2019 it was -0.004. This shows that there was some damage to the overall generosity of people during the pandemic years. It has increased a bit in 2021 but it is still lower than it has been in the past 10 years. 
 
For the more wealthy countries there seems to have been a shift in 2021. Where people seem more interested in being generous. There is only one other time the median was ever this close to zero and it was in 2014 where generosity was above zero at 0.002.  besides that 2021 is the second highest year for generosity with a median of -0.009


In [ ]:
finland = df_all_reports.loc[(df_all_reports['Country name']=='Finland'), :]
denmark = df_all_reports.loc[(df_all_reports['Country name']=='Denmark'), :]
switzerland = df_all_reports.loc[(df_all_reports['Country name']=='Switzerland'), :]
afghanistan = df_all_reports.loc[(df_all_reports['Country name']=='Afghanistan'), :]
rwanda = df_all_reports.loc[(df_all_reports['Country name']=='Rwanda'), :]
zimbabwe = df_all_reports.loc[(df_all_reports['Country name']=='Zimbabwe'), :]

In [ ]:
# Plot 
fig = px.line( title= 'Generosity of diffrent countries over time.', )

#fig.update_layout(yaxis=dict(range=[-0.2,0.35]), xaxis=dict(range=[2005,2021]), plot_bgcolor= '#e6e6e6')
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#f2f2f2', title_text = "Year",)
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#f2f2f2', title_text = "Generosity",)

# Adding countries to graph
fig.add_scatter(x=finland['year'], y=finland['Generosity'], mode='lines', line_color = '#00ff00', showlegend=True, name = 'Finland')
fig.add_scatter(x=denmark['year'], y=denmark['Generosity'], mode='lines', line_color = '#248f24', showlegend=True, name = 'Denmark')
fig.add_scatter(x=switzerland['year'], y=switzerland['Generosity'], mode='lines', line_color = '#00ffa6', showlegend=True, name = 'Switzerland')

fig.add_scatter(x=afghanistan['year'], y=afghanistan['Generosity'], mode='lines', line_color = '#ff5517', showlegend=True, name = 'Afghanistan')
fig.add_scatter(x=rwanda['year'], y=rwanda['Generosity'], mode='lines', line_color = '#0080ff', showlegend=True, name = 'Rwanda')
fig.add_scatter(x=zimbabwe['year'], y=zimbabwe['Generosity'], mode='lines', line_color = '#3300cc', showlegend=True, name = 'Zimbabwe')

fig.show()

The countries depicted above are the 3 happiest countries and three countries that are all in the lowest of happiness. Over all the happier countries feel they are  are less generous in recent years but the slope seems to have started before the pandemic. The less happy countries seem to have a diffrenct perception of their generoisty but the change in slope seems to have started before the pandemic. The intresting observation is that that are all going to the same direction, specificaly between -0.11 and 0.1

# Visulization Of Social Support

In [ ]:
figscatter = px.scatter(df_all_reports,
                        x = 'year', 
                        y = 'Social support', 
                        color = 'Log GDP per capita', 
                        color_continuous_scale=scl,
                        hover_name='Country name',
                        title= 'Social support of diffrent countries over time.')

df_mean_social = []
for y in year1:
  df_year_mean= df_all_reports[(df_all_reports['year']== y)]
  year_mean  = round(df_year_mean['Social support'].mean(), 3)
  df_mean_social.append(year_mean)

figscatter.add_trace(
    go.Scatter(
        x= year1,
        y=df_mean_social,
        name = 'Mean of Social support per year',
        line_color = '#ff5517'
    ))
figscatter.update_layout(plot_bgcolor= '#bfbfbf',legend=dict(yanchor="bottom",xanchor="right",))
figscatter.update_xaxes(title_text = "Year")
figscatter.update_yaxes( title_text = "Social support",)
figscatter.show()

The above scatter plot shows social support over the years and the color is the GDP of each country. Overall you can see that less wealthy countries have worse social support than wealthy countries. During the first pandemic year you can see that there is an increase in social support. which then tapers down in 2021. It is interesting to see during times of crisis, people feel more supported than before. 

In [ ]:
# Plot 
fig = px.line( title= 'Social support of diffrent countries over time.', )

fig.update_layout( xaxis=dict(range=[2005,2021]), plot_bgcolor= '#e6e6e6')
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#f2f2f2', title_text = "Year",)
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#f2f2f2', title_text = "Social support",)

# Adding countries to graph
fig.add_scatter(x=finland['year'], y=finland['Social support'], mode='lines', line_color = '#00ff00', showlegend=True, name = 'Finland')
fig.add_scatter(x=denmark['year'], y=denmark['Social support'], mode='lines', line_color = '#248f24', showlegend=True, name = 'Denmark')
fig.add_scatter(x=switzerland['year'], y=switzerland['Social support'], mode='lines', line_color = '#00ffa6', showlegend=True, name = 'Switzerland')

fig.add_scatter(x=afghanistan['year'], y=afghanistan['Social support'], mode='lines', line_color = '#ff5517', showlegend=True, name = 'Afghanistan')
fig.add_scatter(x=rwanda['year'], y=rwanda['Social support'], mode='lines', line_color = '#0080ff', showlegend=True, name = 'Rwanda')
fig.add_scatter(x=zimbabwe['year'], y=zimbabwe['Social support'], mode='lines', line_color = '#3300cc', showlegend=True, name = 'Zimbabwe')

fig.show()

These six countries are the same we looked at above (Happiest countries vs less happy countries). As you can see from the graph, happy countries are very consistent in their thoughts of social support. Less happy countries fluctuate more. Some data was missing for the poorer countries in 2020. There is an increase from 2019 to 2021.  

In [ ]:
df_mean_pos = []
df_mean_neg = []
df_mean_ladder = []
#Perceptions of corruption
#Freedom to make life choices
df_mean_free = []
df_mean_per = []
for y in year1:
  df_mean_year= df_all_reports[(df_all_reports['year']== y)]
  year_mean  = round((df_mean_year['Life Ladder'].mean()/10), 3) # life ladder is on 10
  df_mean_ladder.append(year_mean)
  
  df_mean_year1= df_all_reports[(df_all_reports['year']== y)]
  year_mean1  = round(df_mean_year1['Positive affect'].mean(), 3) # life ladder is on 10
  df_mean_pos.append(year_mean1)

  df_mean_year2= df_all_reports[(df_all_reports['year']== y)]
  year_mean2  = round(df_mean_year2['Negative affect'].mean(), 3) # life ladder is on 10
  df_mean_neg.append(year_mean2)

  df_mean_year3= df_all_reports[(df_all_reports['year']== y)]
  year_mean3  = round(df_mean_year3['Perceptions of corruption'].mean(), 3) # life ladder is on 10
  df_mean_per.append(year_mean3)

  df_mean_year4= df_all_reports[(df_all_reports['year']== y)]
  year_mean4 = round(df_mean_year4['Freedom to make life choices'].mean(), 3) # life ladder is on 10
  df_mean_free.append(year_mean4)

scl = [[0.0, '#ffffff'],[0.2, '#ccffdd'],[0.4, '#66ff99'], [0.6, '#00e64d'],[0.8, '#009933'],[1.0, '#003311']] # greens
  
fighist = px.scatter (title= "The average of columns over time")
fighist.add_scatter(x=year1, y= df_mean_ladder, showlegend=True,line_color = '#009933', name = 'Life Ladder')
fighist.add_scatter( x=year1, y= df_mean_social, showlegend=True, line_color = '#00e64d', name = 'Social support')
scl = [[0.0, '#ffffff'],[0.2, '#ccffdd'],[0.4, '#66ff99'], [0.6, '#00e64d'],[0.8, '#009933'],[1.0, '#003311']] # greens
fighist.add_scatter(x=year1, y= df_mean_pos, showlegend=True, line_color = '#ccffdd', name = 'Positive affect')
fighist.add_scatter( x=year1, y= df_mean_neg, showlegend=True, line_color = '#03b1fc', name = 'Negative affect')
fighist.add_scatter(x=year1, y= df_mean_free, showlegend=True, line_color = '#03a1fc', name = 'Freedom to make life choices')
fighist.add_scatter( x=year1, y= df_mean_per, showlegend=True, line_color = '#032cfc', name = 'Perceptions of corruption')
fighist.update_layout(plot_bgcolor= '#e6e6e6')
fighist.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#f2f2f2', title_text = "Year",)
fighist.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#f2f2f2', title_text = "Diffrent columns",)

fighist.show()

This last graph shows some other columns over time. This is just to recognize that there are other intresting features in related to pandemic years. For example in 2020 the perception of corruptioin decreased over all. There was also a peak in freedom to make life choices in 2020 as well. 


In conlclusion although the data doesnt state for certain that the cause of changes was do to the pandemic, it probably plays a factor in the changes.


If you have any feedback or comments I would be happy to hear it.
-Rachel